# ओपनबीएनबी एमसीपी सर्वर इंटीग्रेशन के साथ सेमांटिक केर्नेल

यह नोटबुक दिखाती है कि सेमांटिक केर्नेल का उपयोग करके असली ओपनबीएनबी एमसीपी सर्वर के साथ एमसीपीस्टूडियोप्लगइन का उपयोग करके वास्तविक एयरबीएनबी आवासों की खोज कैसे की जा सकती है। एलएलएम एक्सेस के लिए, यह Azure AI Foundry का उपयोग करता है। अपने पर्यावरण वेरिएबल्स सेटअप करने के लिए, आप [सेटअप पाठ](/00-course-setup/README.md) का अनुसरण कर सकते हैं।


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP प्लगइन कनेक्शन बनाना

हम [OpenBnB MCP सर्वर](https://github.com/openbnb-org/mcp-server-airbnb) से MCPStdioPlugin का उपयोग करके कनेक्ट करेंगे। यह सर्वर @openbnb/mcp-server-airbnb पैकेज के माध्यम से Airbnb खोज सुविधा प्रदान करता है।


## क्लाइंट बनाना

इस उदाहरण में, हम Azure AI Foundry का उपयोग अपने LLM एक्सेस के लिए करेंगे। सुनिश्चित करें कि आपके पर्यावरण वेरिएबल्स सही तरीके से सेट किए गए हैं।


## पर्यावरण कॉन्फ़िगरेशन

Azure OpenAI सेटिंग्स कॉन्फ़िगर करें। सुनिश्चित करें कि आपके पास निम्नलिखित पर्यावरण वेरिएबल्स सेट हैं:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP इंटीग्रेशन को समझना

यह नोटबुक **वास्तविक OpenBnB MCP सर्वर** से जुड़ता है, जो असली Airbnb खोज सुविधा प्रदान करता है।

### यह कैसे काम करता है:

1. **MCPStdioPlugin**: MCP सर्वर के साथ मानक इनपुट/आउटपुट संचार का उपयोग करता है  
2. **वास्तविक NPM पैकेज**: `@openbnb/mcp-server-airbnb` को npx के माध्यम से डाउनलोड और चलाता है  
3. **लाइव डेटा**: Airbnb API से वास्तविक संपत्ति डेटा लौटाता है  
4. **फंक्शन डिस्कवरी**: एजेंट MCP सर्वर से उपलब्ध फंक्शन्स को स्वचालित रूप से खोजता है  

### उपलब्ध फंक्शन्स:

OpenBnB MCP सर्वर आमतौर पर निम्नलिखित प्रदान करता है:
- **search_listings** - स्थान और मानदंड के आधार पर Airbnb संपत्तियों की खोज करें  
- **get_listing_details** - विशिष्ट संपत्तियों की विस्तृत जानकारी प्राप्त करें  
- **check_availability** - विशिष्ट तिथियों के लिए उपलब्धता जांचें  
- **get_reviews** - संपत्तियों के लिए समीक्षाएं प्राप्त करें  
- **get_host_info** - संपत्ति के होस्ट की जानकारी प्राप्त करें  

### आवश्यकताएँ:

- आपके सिस्टम पर **Node.js** स्थापित होना चाहिए  
- MCP सर्वर पैकेज डाउनलोड करने के लिए **इंटरनेट कनेक्शन**  
- **NPX** उपलब्ध होना चाहिए (Node.js के साथ आता है)  

### कनेक्शन का परीक्षण:

आप MCP सर्वर को मैन्युअल रूप से चलाकर परीक्षण कर सकते हैं:
```bash
npx -y @openbnb/mcp-server-airbnb
```

यह OpenBnB MCP सर्वर को डाउनलोड और शुरू करेगा, जिससे Semantic Kernel वास्तविक Airbnb डेटा के लिए जुड़ता है।


## OpenBnB MCP सर्वर के साथ एजेंट चलाना

अब हम AI एजेंट को चलाएंगे जो OpenBnB MCP सर्वर से जुड़ता है ताकि स्टॉकहोम में 2 वयस्कों और 1 बच्चे के लिए असली Airbnb आवास खोज सके। खोज मानदंड बदलने के लिए `user_inputs` सूची को संशोधित करने में संकोच न करें।


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

सारांश  
बधाई हो! आपने एक AI एजेंट सफलतापूर्वक बनाया है जो Model Context Protocol (MCP) का उपयोग करके वास्तविक दुनिया के आवास खोज के साथ एकीकृत होता है:

प्रयुक्त तकनीकें:  
- Semantic Kernel - Azure OpenAI के साथ बुद्धिमान एजेंट बनाने के लिए  
- Azure AI Foundry - LLM क्षमताओं और चैट पूर्णता के लिए  
- MCP (Model Context Protocol) - मानकीकृत टूल एकीकरण के लिए  
- OpenBnB MCP Server - वास्तविक Airbnb खोज कार्यक्षमता के लिए  
- Node.js/NPX - बाहरी MCP सर्वर चलाने के लिए  

आपने क्या सीखा:  
- MCP एकीकरण: Semantic Kernel एजेंट्स को बाहरी MCP सर्वरों से जोड़ना  
- रीयल-टाइम डेटा एक्सेस: लाइव APIs के माध्यम से वास्तविक Airbnb संपत्तियों की खोज  
- प्रोटोकॉल संचार: एजेंट और MCP सर्वर के बीच stdio संचार का उपयोग  
- फंक्शन डिस्कवरी: MCP सर्वरों से उपलब्ध फंक्शन्स को स्वचालित रूप से खोजना  
- स्ट्रीमिंग प्रतिक्रियाएं: रीयल-टाइम में फंक्शन कॉल्स को कैप्चर और लॉग करना  
- HTML रेंडरिंग: एजेंट प्रतिक्रियाओं को स्टाइल्ड टेबल्स और इंटरएक्टिव डिस्प्ले के साथ फॉर्मेट करना  

अगले कदम:  
- अतिरिक्त MCP सर्वरों को एकीकृत करें (मौसम, उड़ानें, रेस्तरां)  
- MCP और A2A प्रोटोकॉल को मिलाकर एक मल्टी-एजेंट सिस्टम बनाएं  
- अपने डेटा स्रोतों के लिए कस्टम MCP सर्वर बनाएं  
- सत्रों के बीच स्थायी बातचीत मेमोरी लागू करें  
- MCP सर्वर ऑर्केस्ट्रेशन के साथ एजेंट को Azure Functions पर तैनात करें  
- उपयोगकर्ता प्रमाणीकरण और बुकिंग क्षमताएं जोड़ें  

MCP आर्किटेक्चर के मुख्य लाभ:  
- मानकीकरण: बाहरी टूल्स से AI एजेंट्स को जोड़ने के लिए एक सार्वभौमिक प्रोटोकॉल  
- रीयल-टाइम डेटा: विभिन्न सेवाओं से लाइव, अद्यतन जानकारी तक पहुंच  
- विस्तारशीलता: नए डेटा स्रोतों और टूल्स को आसानी से एकीकृत करना  
- इंटरऑपरेबिलिटी: विभिन्न AI फ्रेमवर्क्स और एजेंट प्लेटफॉर्म्स पर काम करता है  
- जिम्मेदारियों का विभाजन: AI लॉजिक और बाहरी डेटा एक्सेस के बीच स्पष्ट अंतर



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को आधिकारिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम जिम्मेदार नहीं हैं।
